# 1. Импорт данных и библиотек

In [2]:
import pandas as pd
import numpy as np

DATA_PATH = 'C:/Users/kokontsev/Desktop/Sleep-Quality-Prediction/data/processed/'

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)

In [3]:
df = pd.read_csv(DATA_PATH + "sleep_activity_daily.csv")

df.head()

,user_id,sleep_date,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed,SleepEfficiency,activity_date,TotalSteps,TotalDistance,Calories,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes
0,1503960366,2016-04-12,1,327,346,0.945087,2016-04-11,13162,8.50,1985,25,13,328,728
1,1503960366,2016-04-13,2,384,407,0.943489,2016-04-12,10735,6.97,1797,21,19,217,776
2,1503960366,2016-04-15,1,412,442,0.932127,2016-04-14,9762,6.28,1745,29,34,209,726
3,1503960366,2016-04-16,2,340,367,0.926431,2016-04-15,12669,8.16,1863,36,10,221,773
4,1503960366,2016-04-17,1,700,712,0.983146,2016-04-16,9705,6.48,1728,38,20,164,539


# 2. Базовая очистка данных
## 2.1. Удаление наблюдений без данных сна

In [4]:
df = df.dropna(subset=["TotalMinutesAsleep", "TotalTimeInBed", "SleepEfficiency"])

## 2.2. Удаление явно некорректных значений сна

In [5]:
df = df[
    (df["TotalMinutesAsleep"] > 0) &
    (df["TotalTimeInBed"] > 0) &
    (df["SleepEfficiency"] <= 1)
]


# 3. Формирование новых признаков (Feature Engineering)

## 3.1. Суммарная активность

In [7]:
df["TotalActiveMinutes"] = (
    df["VeryActiveMinutes"] +
    df["FairlyActiveMinutes"] +
    df["LightlyActiveMinutes"]
)

## 3.2. Доли активности

In [8]:
df["ActiveMinutesShare"] = (
    df["TotalActiveMinutes"] /
    (df["TotalActiveMinutes"] + df["SedentaryMinutes"])
)

## 3.3. Интенсивность активности

In [10]:
df["CaloriesPerStep"] = df["Calories"] / df["TotalSteps"].replace(0, np.nan)
df["CaloriesPerStep"] = df["CaloriesPerStep"].fillna(0)

## 3.4. Относительная доля интенсивной активности

In [11]:
df["VeryActiveShare"] = (
    df["VeryActiveMinutes"] / df["TotalActiveMinutes"].replace(0, np.nan)
)
df["VeryActiveShare"] = df["VeryActiveShare"].fillna(0)


# 4. Пользовательские агрегаты

## 4.1. Средняя эффективность сна пользователя

In [ ]:
user_sleep_mean = (
    df.groupby("user_id")["SleepEfficiency"]
      .mean()
      .rename("UserMeanSleepEfficiency")
)

df = df.merge(user_sleep_mean, on="user_id", how="left")

## 4.2. Отклонение от среднего по пользователю

In [ ]:
df["SleepEfficiencyDeviation"] = (
    df["SleepEfficiency"] - df["UserMeanSleepEfficiency"]
)

# 5. Подготовка финального набора признаков

## 5.1. Отбор признаков

In [14]:
feature_cols = [
    "TotalSteps",
    "Calories",
    "TotalActiveMinutes",
    "ActiveMinutesShare",
    "VeryActiveShare",
    "SedentaryMinutes",
    "CaloriesPerStep"
]

target_col = "SleepEfficiency"


## 5.2. Финальный датасет

In [15]:
X = df[feature_cols]
y = df[target_col]

X.shape, y.shape


((413, 7), (413,))

## 6. Сохранение датасета для моделирования

In [16]:
df.to_csv(DATA_PATH + "sleep_activity_fe.csv", index=False)

# 7. Промежуточные выводы
**Что сделано:**
- выполнена очистка данных сна;
- обработаны пропуски в признаках активности;
- сформированы агрегированные и относительные признаки;
- учтены индивидуальные особенности пользователей.

**Результат:** Получен финальный датасет, пригодный для построения и сравнения моделей прогнозирования качества сна.